# Etherscan verified contract database search

# Concatenate databases

In [1]:
import csv
import os
import glob
import pandas as pd
import tqdm as tqdm

In [2]:
# Look for all csv files
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

In [3]:
# Note that if you run this multiple times it will add combined_csv back to the csv file.
# ! Run this only once !
if not os.path.exists("combined_csv.csv"):
    combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames])
    combined_csv.to_csv("combined_csv.csv", index=False, encoding='utf-8-sig')


In [4]:
data = pd.read_csv(r'combined_csv.csv')
data

,Txhash,ContractAddress,ContractName
0,0xc439780ceb4726c5f1827d3f4a5d4c84b4deab83de5b...,0xf8D631FC499276F794f8D5b8F462C5b11A1d1Eef,StandardERC20
1,0xfda7fab6b89cb7a8fd827c4da118877ee5786d798845...,0x63c867e40ce5c278d153149060ef9b1d8a58c7ff,EthereumInu
2,0x066accd2a165a7b8a3d9f881f148d3a447ad047c17e4...,0x687c7cb36bb610bae16d2b4ec0f6584f29126767,PEPECOIN
3,0x0ff96b116a4a1cc9dd5fdb9e841e9e745e0126ef3a0f...,0x9287AEFe51047Ef43F68612f80BAC17745b23AeC,XRIToken
4,0x25f9d0716a5f99517b5b5ed7d78a92d007a3770363d1...,0xaebfe92a9f5e874bd680946b3a394e5165072569,ERC721Helper
...,...,...,...
63094,0x79a9f27073f953b832fd511015cc5f75a7eeac747e23...,0x6A858592fC4cBdf432Fc9A1Bc8A0422B99330bdF,DSValue
63095,0x311ae8809d6b7313f4d2b19b9bed94714ddeca13d141...,0x3f90622f3df1fe37e6233a8446a835ab032bb941,Registry
63096,0x021e9779d067aee34aa2ab93298f824f27d4b58c8e30...,0x2ce5a05aa98509632aa04a036337d0f8305e9e64,CatsCorporation
63097,0x54e6c6da32c28c17d818595d096f2fc927a8c7bc352a...,0x122Fb3Cb3f7939f58C808c0f267Dfa0eCAd0174C,NoumenaFloral


In [5]:
import requests
# Import your own apikey
apikey = '5NH6QCI59NUB74G9FY712QEHGIHZ4B4BBR'

In [13]:
def api_call(row):
    val = ""
    contractaddress = data['ContractAddress'][row]
    name = data['ContractName'][row]
    url = f'https://api.etherscan.io/api?module=contract&action=getsourcecode&address={contractaddress}&apikey={apikey}'
    response = requests.get(url)
    response.raise_for_status()
    if response.status_code != 204:

        try:
            result = response.json()['result']
            val = result[0].get('SourceCode')
        except ValueError:
         val = "Value Error"

    return name, contractaddress, val


    #
    # call_count = val.count('.call(')
    # send_count = val.count('.send(')
    # transfer_count = val.count('.transfer(')


In [7]:
def write_to_csv(data, filename):
    with open(filename, 'w') as f:
        writer = csv.writer(f, delimiter=",")
        writer.writerow(['ContractName','ContractAddress','Source'])
        for row in data:
            writer.writerows(row)

In [8]:
from multiprocessing import Pool, cpu_count
from p_tqdm import p_map
def multi_call(func, args):
    results = []
    with Pool(cpu_count()) as p:
        completed = p_map(func, args, **{"num_cpus":cpu_count()})
    results.append(completed)
    return results

In [9]:
def run_apply_async_multiprocessing(func, argument_list, num_processes):

    pool = Pool(processes=num_processes)

    jobs = [pool.apply_async(func=func, args=(*argument,)) if isinstance(argument, tuple) else pool.apply_async(func=func, args=(argument,)) for argument in argument_list]
    pool.close()
    result_list_tqdm = []
    for job in tqdm.tqdm(jobs):
        print(job.get()[1])

        result_list_tqdm.append(dict)

    return result_list_tqdm

In [10]:
import random
argument_list = [i for i in range(len(data))]
arg_list_1 = argument_list[:10000]
arg_list_2 = argument_list[10000:20000]
arg_list_3 = argument_list[20000:30000]
arg_list_4 = argument_list[30000:40000]
arg_list_5 = argument_list[40000:50000]
arg_list_6 = argument_list[50000:]

In [11]:
# Result 1
result1 = multi_call(api_call, arg_list_1)
write_to_csv(result1, 'contracts_1.csv')

  0%|          | 0/10000 [00:00<?, ?it/s]

In [14]:
# Result 2
result2 = multi_call(api_call, arg_list_2)
write_to_csv(result2, 'contracts_2.csv')

  0%|          | 0/10000 [00:00<?, ?it/s]

In [15]:
# Result 3
result3 = multi_call(api_call, arg_list_3)
write_to_csv(result3, 'contracts_3.csv')

  0%|          | 0/10000 [00:00<?, ?it/s]

In [16]:
# Result 4
result4 = multi_call(api_call, arg_list_4)
write_to_csv(result4, 'contracts_4.csv')

  0%|          | 0/10000 [00:00<?, ?it/s]

In [ ]:
# Result 5
result5 = multi_call(api_call, arg_list_5)
write_to_csv(result5, 'contracts_5.csv')

In [ ]:
# Result 6
result6 = multi_call(api_call, arg_list_6)
write_to_csv(result6, 'contracts_6.csv')

In [46]:
# Functionality for multi threading, did not speed up results

# import requests
# import uuid
# from concurrent.futures import ThreadPoolExecutor, as_completed
#
# def runner():
#     threads = []
#     with tqdm.tqdm(total=len(data['ContractAddress'])) as pbar:
#         with ThreadPoolExecutor(max_workers=10) as executor:
#             for contract in range(len(data['ContractAddress'])):
#                 threads.append((executor.submit(api_call,contract)))
#             for task in as_completed(threads):
#                 print(task.result())
#                 pbar.update(1)
# runner()

  0%|          | 0/63099 [00:00<?, ?it/s]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

  0%|          | 0/63099 [00:08<?, ?it/s]


KeyboardInterrupt: 